In [15]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
from sklearn.model_selection import train_test_split

# Load datasets

X_train = pd.read_parquet(
    r"D:\Duke\IDS705 ML\Final_project\enivornmental_impact_of_aviation\Train-test-validation_with_outliers/X_train.parquet"
)
X_test = pd.read_parquet(
    r"D:\Duke\IDS705 ML\Final_project\enivornmental_impact_of_aviation\Train-test-validation_with_outliers/X_test.parquet"
)
X_val = pd.read_parquet(
    r"D:\Duke\IDS705 ML\Final_project\enivornmental_impact_of_aviation\Train-test-validation_with_outliers/X_val.parquet"
)
y_train = pd.read_parquet(
    r"D:\Duke\IDS705 ML\Final_project\enivornmental_impact_of_aviation\Train-test-validation_with_outliers/y_train.parquet"
).squeeze()
y_test = pd.read_parquet(
    r"D:\Duke\IDS705 ML\Final_project\enivornmental_impact_of_aviation\Train-test-validation_with_outliers/y_test.parquet"
).squeeze()
y_val = pd.read_parquet(
    r"D:\Duke\IDS705 ML\Final_project\enivornmental_impact_of_aviation\Train-test-validation_with_outliers/y_val.parquet"
).squeeze()

In [16]:
df = pd.read_parquet(
    r"D:\Duke\IDS705 ML\Final_project\enivornmental_impact_of_aviation\cleaned_aviation_data_with_outliers_v4.parquet"
)
df

,airline_iata,acft_icao,acft_class,seymour_proxy,source,seats,n_flights,iata_departure,iata_arrival,departure_lon,...,distance_km,ask,rpk,fuel_burn_seymour,fuel_burn,co2,domestic,log_co2,log_distance,co2_per_distance
0,1SQ,PA32,PP,P28B,BTS,62112.0,11466.5,SPN,TIQ,145.729004,...,17.765225,1.103434e+06,9.092294e+05,20.486379,2.349071e+05,7.423063e+05,1,13.517519,2.932005,41784.233355
1,1SQ,PA32,PP,P28B,BTS,62112.0,11466.5,TIQ,SPN,145.619003,...,17.765225,1.103434e+06,9.092294e+05,20.486379,2.349071e+05,7.423063e+05,1,13.517519,2.932005,41784.233355
2,HA,B712,NB,B712,BTS,1246720.0,9740.0,HNL,OGG,-157.924228,...,162.001417,2.019704e+08,1.664236e+08,1305.895783,1.271942e+07,4.019338e+07,1,17.509213,5.093759,248105.130283
3,HA,B712,NB,B712,BTS,1246720.0,9740.0,OGG,HNL,-156.431212,...,162.001417,2.019704e+08,1.664236e+08,1305.895783,1.271942e+07,4.019338e+07,1,17.509213,5.093759,248105.130283
4,HA,B712,NB,B712,BTS,879296.0,6869.5,HNL,KOA,-157.924228,...,262.778946,2.310605e+08,1.903938e+08,1567.819046,1.077013e+07,3.403362e+07,1,17.342859,5.575111,129514.257569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291639,ACN,None,None,None,ANAC,4.5,4.0,SXO,CGB,-50.689602,...,736.539053,3.314426e+03,2.731087e+03,4223.750606,1.010269e+02,3.192451e+02,1,5.769087,6.603319,0.433439
291640,ACN,None,None,None,ANAC,4.5,4.0,QDV,CGH,-46.943921,...,57.645994,2.594070e+02,2.137513e+02,4223.750606,4.807282e+01,1.519101e+02,1,5.029850,4.071519,2.635224
291643,ACN,None,None,None,ANAC,4.5,4.0,QDV,POO,-46.943921,...,153.120924,6.890442e+02,5.677724e+02,4223.750606,5.527809e+01,1.746788e+02,1,5.168657,5.037738,1.140790
291644,ACN,None,None,None,ANAC,4.5,4.0,RRJ,QDV,-43.372194,...,366.577348,1.649598e+03,1.359269e+03,4223.750606,7.167492e+01,2.264927e+02,1,5.427118,5.906934,0.617858


In [13]:
(X_train["domestic_x"] == X_train["domestic_y"]).all()

KeyError: 'domestic_x'

In [ ]:
# X_train["domestic"] = X_train["domestic_x"]  # or "domestic_y"
# X_train.drop(columns=["domestic_x", "domestic_y"], inplace=True)

# X_train["domestic"] = X_train["domestic"].astype("category")

# X_val["domestic"] = X_val["domestic_x"]
# X_val.drop(columns=["domestic_x", "domestic_y"], inplace=True)
# X_val["domestic"] = X_val["domestic"].astype("category")

# X_test["domestic"] = X_test["domestic_x"]
# X_test.drop(columns=["domestic_x", "domestic_y"], inplace=True)
# X_test["domestic"] = X_test["domestic"].astype("category")

KeyError: 'domestic_x'

In [19]:
for i in [X_train, X_test, X_val]:
    i["is_widebody"] = i["is_widebody"].astype("category")
    i["domestic"] = i["domestic"].astype("category")
    i["is_narrowbody"] = i["is_narrowbody"].astype("category")

In [20]:
original_cats = df[
    [
        "acft_class",
        "departure_country",
        "arrival_country",
        "departure_continent",
        "arrival_continent",
        # "domestic",
        # "is_widebody",
        # "is_narrowbody",
    ]
]

In [21]:
# Align index to encoded splits
X_train = X_train.merge(original_cats, left_index=True, right_index=True)
X_val = X_val.merge(original_cats, left_index=True, right_index=True)
X_test = X_test.merge(original_cats, left_index=True, right_index=True)

In [ ]:
X_train["domestic"] = X_train["domestic"].astype(bool)
X_val["domestic"] = X_val["domestic"].astype(bool)
X_test["domestic"] = X_test["domestic"].astype(bool)

In [ ]:
numeric_features = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()
numeric_features

['seats',
 'n_flights',
 'ask',
 'rpk',
 'fuel_burn',
 'fuel_burn_per_seat',
 'ask_per_seat',
 'ask_per_flight']

In [ ]:
encoded_only_cols = [
    col
    for col in X_test.columns
    if col not in original_cats.columns and col not in numeric_features
]
# X_train.drop(columns=encoded_only_cols, inplace=True)
X_test.drop(columns=encoded_only_cols, inplace=True)
X_val.drop(columns=encoded_only_cols, inplace=True)

In [39]:
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

categorical_cols = [
    "acft_class",
    "departure_country",
    "arrival_country",
    "departure_continent",
    "arrival_continent",
    # "domestic",
    # "is_widebody",
    # "is_narrowbody",
    # "same_continent",
    # "same_country",
]

# # If using original DataFrame (with categoricals):
# X = df[[
#     "acft_class", "n_flights", "departure_country", "departure_continent",
#     "arrival_country", "arrival_continent", "domestic", "ask", "rpk",
#     "fuel_burn", "same_continent", "same_country",
#     "ask_per_seat", "ask_per_flight", "fuel_burn_per_seat", "is_widebody", "is_narrowbody"
# ]]

# y = df["co2_per_distance"]

# # Split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create CatBoost Pool (handles categoricals internally)
train_pool = Pool(X_train, y_train, cat_features=categorical_cols)
test_pool = Pool(X_test, y_test, cat_features=categorical_cols)

# Initialize and train
model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.1,
    depth=8,
    loss_function="RMSE",
    early_stopping_rounds=50,
    verbose=100,
)

model.fit(train_pool, eval_set=test_pool)

# Predict and evaluate
y_pred = model.predict(X_test)

print("Test Set Evaluation:")
print("R²:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

TypeError: must be real number, not NoneType